In [13]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [14]:
params = {
    #Original data path
    'data' : 'data/2025-07-10-LE_Lisket_0_15_AMBI_DEF.xlsx',
    #Columns from the original dataset that are not needed for the analysis 
    'to_drop_cols' : ['Date_Ambitus', 'GR_Gender', 'G_S', 'Animal', 'EAT_E_Nr', 'EAT_I_Nr', 'EAT_TOT_Nr', 'Expl_E_BEF_Nr', 'Expl_I_BEF_Nr', 'Expl_E_I_BEF_Nr', 'Expl_E_AFT_Nr', 'Expl_I_AFT_Nr', 
                      'Expl_E_I_AFT_Nr', 'Expl_E_BEF_T', 'Expl_I_BEF_T', 'Expl_E_I_BEF_T', 'Expl_E_AFT_T', 'Expl_I_AFT_T', 'Expl_E_I_AFT_T', 'LAT_E', 'LAT_I', 'LAT_E_I', 'Expl_REP_E_BEF_Nr', 'Expl_REP_I_BEF_Nr',
                      'Expl_REP_BEF_Nr', 'Expl_REP_E_AFT_Nr', 'Expl_REP_I_AFT_Nr', 'Expl_REP_AFT_Nr', 'Loco_BEF', 'L_C_Tot', 'WM_ERROR_E', 'WM_ERROR_I', 'WM_ERROR_EI', 'WM_ERR_RW', 'WM_ERR_NRW',
                      'Expl_E_AFT_T_Calc','Expl_I_AFT_T_Calc','Expl_E_I_AFT_T_Calc','Expl_E_BEF_Calc_T','Expl_I_BEF_Calc_T','Expl_E_I_BEF_Calc_T','Expl_E_AFT_Calc','Expl_I_AFT_Calc','Expl_E_I_AFT_Calc',
                      'Expl_REP_E_AFT_Nr_Calc',	'Expl_REP_I_AFT_Nr_Calc',	'Expl_REP_AFT_Nr_Calc'],
    'no_merge_cols': ['Group', 'NR', 'Generation', 'Trials', 'Paradigm']
}

In [15]:
#Helper functions
def gen_id(data):
    """Create a unique identifier from group, number, generation.

    Args:
        data (pd.DataFrame): raw data
    Returns:
        pd.DataFrame: data with an additional 'id' column
    """
    data['id'] = data['Group'].astype(str) + '_' + data['NR'].astype(str) + '_' + data['Generation'].astype(str)
    return data

def make_rat_one_row(data):
    """From 4 trials converts each rat into a single row.

    Args:
        data (pd.DataFrame): raw data
    Returns:
        pd.DataFrame: data with each rat as a single row
    """
    re_df = pd.DataFrame()
    for id in data['id'].unique():
        temp = data[data['id'] == id]
        to_merge = temp.loc[temp['Trials'] == 1]
        for i in range(2, 5):
            tmp = temp.loc[temp['Trials'] == i].drop(columns=params['no_merge_cols'])
            to_merge = pd.merge(to_merge, tmp, how='inner', on="id", suffixes=('', f'_{i}'))

        re_df = pd.concat([re_df, to_merge], ignore_index=True)
    return re_df
        

In [16]:
excel_data = pd.read_excel(params['data'])
excel_data = excel_data.loc[excel_data['Animal'] != 'Sorszam']
excel_data = excel_data.drop(columns=params['to_drop_cols'], axis=1)
excel_data = gen_id(excel_data)
excel_data

,Generation,Season,Separation,Paradigm,Year,NR,Group,Gender,Trials,EAT_E_%,...,Expl_I_BEF_Loco_ratio,Expl_EI_BEF_Loco_ratio,Expl_E_TOT_Loco_ratio,Expl_I_TOT_Loco_ratio,Expl_E_I_TOT_Loco_ratio,Eff_Expl_E,Eff_Expl_I,Eff_Expl_EI,Eff_Expl_All,id
1,0,Autumn,2019-08-26,1.0,0.0,1,Lisket,Male,1.0,100.0,...,1.076923,2.923077,1.846154,1.076923,2.923077,1.000,0.75,0.875000,0.8750,Lisket_1_0
2,0,Autumn,2019-08-26,1.0,0.0,1,Lisket,Male,2.0,100.0,...,1.857143,3.428571,1.500000,2.100000,3.600000,1.000,1.00,1.000000,1.0000,Lisket_1_0
3,0,Autumn,2019-08-26,2.0,0.0,1,Lisket,Male,3.0,NaN,...,1.800000,2.800000,0.769231,1.000000,1.769231,NaN,1.00,0.533333,1.0000,Lisket_1_0
4,0,Autumn,2019-08-26,2.0,0.0,1,Lisket,Male,4.0,NaN,...,3.166667,4.833333,1.857143,2.357143,4.214286,NaN,1.00,0.500000,1.0000,Lisket_1_0
5,0,Autumn,2019-08-26,1.0,0.0,2,Lisket,Male,1.0,87.5,...,0.916667,2.250000,1.333333,0.916667,2.250000,1.000,1.00,1.000000,1.0000,Lisket_2_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,15,Winter,2024-12-30,2.0,6.0,98,LE,Male,4.0,NaN,...,1.600000,2.500000,0.968750,1.343750,2.312500,NaN,1.00,0.533333,1.0000,LE_98_15
5365,15,Winter,2024-12-30,1.0,6.0,99,LE,Female,1.0,87.5,...,0.520000,1.120000,0.600000,0.520000,1.120000,0.875,1.00,0.937500,0.9375,LE_99_15
5366,15,Winter,2024-12-30,1.0,6.0,99,LE,Female,2.0,100.0,...,1.200000,2.400000,0.846154,0.807692,1.653846,1.000,1.00,1.000000,1.0000,LE_99_15
5367,15,Winter,2024-12-30,2.0,6.0,99,LE,Female,3.0,NaN,...,1.750000,2.875000,0.971429,0.800000,1.771429,NaN,1.00,0.533333,1.0000,LE_99_15


In [17]:
#Replace NaN values with -1
excel_data = excel_data.replace(np.nan, -1)
excel_data = excel_data.replace(' ', -1)
excel_data

,Generation,Season,Separation,Paradigm,Year,NR,Group,Gender,Trials,EAT_E_%,...,Expl_I_BEF_Loco_ratio,Expl_EI_BEF_Loco_ratio,Expl_E_TOT_Loco_ratio,Expl_I_TOT_Loco_ratio,Expl_E_I_TOT_Loco_ratio,Eff_Expl_E,Eff_Expl_I,Eff_Expl_EI,Eff_Expl_All,id
1,0,Autumn,2019-08-26 00:00:00,1.0,0.0,1,Lisket,Male,1.0,100.0,...,1.076923,2.923077,1.846154,1.076923,2.923077,1.000,0.75,0.875000,0.8750,Lisket_1_0
2,0,Autumn,2019-08-26 00:00:00,1.0,0.0,1,Lisket,Male,2.0,100.0,...,1.857143,3.428571,1.500000,2.100000,3.600000,1.000,1.00,1.000000,1.0000,Lisket_1_0
3,0,Autumn,2019-08-26 00:00:00,2.0,0.0,1,Lisket,Male,3.0,-1.0,...,1.800000,2.800000,0.769231,1.000000,1.769231,-1.000,1.00,0.533333,1.0000,Lisket_1_0
4,0,Autumn,2019-08-26 00:00:00,2.0,0.0,1,Lisket,Male,4.0,-1.0,...,3.166667,4.833333,1.857143,2.357143,4.214286,-1.000,1.00,0.500000,1.0000,Lisket_1_0
5,0,Autumn,2019-08-26 00:00:00,1.0,0.0,2,Lisket,Male,1.0,87.5,...,0.916667,2.250000,1.333333,0.916667,2.250000,1.000,1.00,1.000000,1.0000,Lisket_2_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,15,Winter,2024-12-30 00:00:00,2.0,6.0,98,LE,Male,4.0,-1.0,...,1.600000,2.500000,0.968750,1.343750,2.312500,-1.000,1.00,0.533333,1.0000,LE_98_15
5365,15,Winter,2024-12-30 00:00:00,1.0,6.0,99,LE,Female,1.0,87.5,...,0.520000,1.120000,0.600000,0.520000,1.120000,0.875,1.00,0.937500,0.9375,LE_99_15
5366,15,Winter,2024-12-30 00:00:00,1.0,6.0,99,LE,Female,2.0,100.0,...,1.200000,2.400000,0.846154,0.807692,1.653846,1.000,1.00,1.000000,1.0000,LE_99_15
5367,15,Winter,2024-12-30 00:00:00,2.0,6.0,99,LE,Female,3.0,-1.0,...,1.750000,2.875000,0.971429,0.800000,1.771429,-1.000,1.00,0.533333,1.0000,LE_99_15


In [18]:
#Save the modified data to a new CSV file
excel_data.to_csv('data/LE_Lisket_0_15_AMBI_DEF_stat.csv', index=False)

In [19]:
#Create a single row for each rat and unifie names
one_rowed_data = make_rat_one_row(excel_data)
one_rowed_data = one_rowed_data.replace('male', 'Male')
one_rowed_data = one_rowed_data.replace('female', 'Female')
one_rowed_data

,Generation,Season,Separation,Paradigm,Year,NR,Group,Gender,Trials,EAT_E_%,...,Expl_E_BEF_Loco_ratio_4,Expl_I_BEF_Loco_ratio_4,Expl_EI_BEF_Loco_ratio_4,Expl_E_TOT_Loco_ratio_4,Expl_I_TOT_Loco_ratio_4,Expl_E_I_TOT_Loco_ratio_4,Eff_Expl_E_4,Eff_Expl_I_4,Eff_Expl_EI_4,Eff_Expl_All_4
0,0,Autumn,2019-08-26 00:00:00,1.0,0.0,1,Lisket,Male,1.0,100.0,...,1.666667,3.166667,4.833333,1.857143,2.357143,4.214286,-1.0,1.0,0.500000,1.0
1,0,Autumn,2019-08-26 00:00:00,1.0,0.0,2,Lisket,Male,1.0,87.5,...,2.666667,5.333333,8.000000,1.529412,2.117647,3.647059,-1.0,1.0,0.571429,1.0
2,0,Autumn,2019-08-26 00:00:00,1.0,0.0,3,Lisket,Male,1.0,100.0,...,3.000000,7.000000,10.000000,2.076923,3.076923,5.153846,-1.0,1.0,0.571429,1.0
3,0,Autumn,2019-08-26 00:00:00,1.0,0.0,4,LE,Male,1.0,87.5,...,4.500000,8.500000,13.000000,2.818182,3.727273,6.545455,-1.0,1.0,0.571429,1.0
4,0,Autumn,2019-08-26 00:00:00,1.0,0.0,5,Lisket,Male,1.0,50.0,...,2.666667,5.333333,8.000000,1.888889,3.222222,5.111111,-1.0,1.0,0.533333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,15,Winter,2024-12-30 00:00:00,1.0,6.0,95,LE,Male,1.0,87.5,...,2.000000,3.166667,5.166667,1.105263,1.578947,2.684211,-1.0,1.0,0.500000,1.0
1338,15,Winter,2024-12-30 00:00:00,1.0,6.0,96,LE,Male,1.0,100.0,...,1.166667,2.666667,3.833333,1.260870,1.565217,2.826087,-1.0,1.0,0.571429,1.0
1339,15,Winter,2024-12-30 00:00:00,1.0,6.0,97,LE,Male,1.0,100.0,...,0.500000,1.750000,2.250000,1.350000,1.450000,2.800000,-1.0,1.0,0.666667,1.0
1340,15,Winter,2024-12-30 00:00:00,1.0,6.0,98,LE,Male,1.0,100.0,...,0.900000,1.600000,2.500000,0.968750,1.343750,2.312500,-1.0,1.0,0.533333,1.0


In [20]:
#Drop columns that are not needed for the analysis
columns_to_drop = one_rowed_data.filter(regex='Group_|Gender_|Season_|Trials_|NR_|Year_|Paradigm_|Generation_|Unnamed').columns
one_rowed_data = one_rowed_data.drop(columns=columns_to_drop)
#Drop columns that are only contains -1 values
one_rowed_data = one_rowed_data.loc[:, (one_rowed_data != -1).any(axis=0)]
one_rowed_data

,Generation,Season,Separation,Paradigm,Year,NR,Group,Gender,Trials,EAT_E_%,...,A_E_4,Expl_E_BEF_Loco_ratio_4,Expl_I_BEF_Loco_ratio_4,Expl_EI_BEF_Loco_ratio_4,Expl_E_TOT_Loco_ratio_4,Expl_I_TOT_Loco_ratio_4,Expl_E_I_TOT_Loco_ratio_4,Eff_Expl_I_4,Eff_Expl_EI_4,Eff_Expl_All_4
0,0,Autumn,2019-08-26 00:00:00,1.0,0.0,1,Lisket,Male,1.0,100.0,...,27.586207,1.666667,3.166667,4.833333,1.857143,2.357143,4.214286,1.0,0.500000,1.0
1,0,Autumn,2019-08-26 00:00:00,1.0,0.0,2,Lisket,Male,1.0,87.5,...,33.333333,2.666667,5.333333,8.000000,1.529412,2.117647,3.647059,1.0,0.571429,1.0
2,0,Autumn,2019-08-26 00:00:00,1.0,0.0,3,Lisket,Male,1.0,100.0,...,40.000000,3.000000,7.000000,10.000000,2.076923,3.076923,5.153846,1.0,0.571429,1.0
3,0,Autumn,2019-08-26 00:00:00,1.0,0.0,4,LE,Male,1.0,87.5,...,30.769231,4.500000,8.500000,13.000000,2.818182,3.727273,6.545455,1.0,0.571429,1.0
4,0,Autumn,2019-08-26 00:00:00,1.0,0.0,5,Lisket,Male,1.0,50.0,...,33.333333,2.666667,5.333333,8.000000,1.888889,3.222222,5.111111,1.0,0.533333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,15,Winter,2024-12-30 00:00:00,1.0,6.0,95,LE,Male,1.0,87.5,...,25.806452,2.000000,3.166667,5.166667,1.105263,1.578947,2.684211,1.0,0.500000,1.0
1338,15,Winter,2024-12-30 00:00:00,1.0,6.0,96,LE,Male,1.0,100.0,...,34.782609,1.166667,2.666667,3.833333,1.260870,1.565217,2.826087,1.0,0.571429,1.0
1339,15,Winter,2024-12-30 00:00:00,1.0,6.0,97,LE,Male,1.0,100.0,...,44.444444,0.500000,1.750000,2.250000,1.350000,1.450000,2.800000,1.0,0.666667,1.0
1340,15,Winter,2024-12-30 00:00:00,1.0,6.0,98,LE,Male,1.0,100.0,...,32.000000,0.900000,1.600000,2.500000,0.968750,1.343750,2.312500,1.0,0.533333,1.0


In [21]:
one_rowed_data.drop(columns=['Trials'], inplace=True)
one_rowed_data.to_csv("data/LE_Lisket_0_15_AMBI_DEF_ml.csv", index=False)

In [22]:
#Reading in excel, to see if all the columns have descripotion
def_columns = pd.read_excel(params['data'], sheet_name='Definition')
#Check which columns are missing description from excel_data dataframe
missing_descriptions = [col for col in excel_data.columns if col not in def_columns['Új def'].values]
missing_descriptions

['Generation',
 'Season',
 'Separation',
 'Paradigm',
 'Year',
 'NR',
 'Group',
 'Gender',
 'Trials',
 'Unnamed: 79',
 'Unnamed: 81',
 'id']